In [12]:
import statistics
import queue
from collections import namedtuple

In [13]:
Item = namedtuple("Item", ['index', 'value', 'weight', 'value_per_weight'])
Node = namedtuple("Node", ['direction', 'index', 'value', 'weight', 'estimate', 'excluded_paths'])

In [14]:
def get_children(node_value, capacity, node_estimate, item, items, paths):
        
    optimistic_value = 0
    total_weight =  MAX_CAPACITY

    filtered_items = [ele for ele in items if ele.index not in paths+[item.index]]
    
    for ele in sorted(filtered_items, key=lambda x: x.value_per_weight, reverse=True):
        if total_weight-ele.weight > 0 :
            optimistic_value += ele.value
            total_weight -= ele.weight  
        else:
            optimistic_value += total_weight * ele.value_per_weight
            break
            
    optimistic_value = int(optimistic_value)
    
    if capacity-item.weight >= 0 :
        left_node = Node('left', item.index, node_value+item.value, capacity-item.weight, node_estimate, paths)
        right_node = Node('right', item.index, node_value, capacity, optimistic_value, paths+[item.index]) #node_estimate-item.value) 
    else:
        left_node = Node('left', item.index, -1, capacity-item.weight, -1, paths)
        right_node = Node('right', item.index, node_value, capacity, optimistic_value, paths+[item.index]) #node_estimate-item.value)

    return left_node, right_node

<!-- ## Depth First Search -->

In [15]:
with open('./data/ks_4_0', 'r') as input_data_file:
    input_data = input_data_file.read()

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
total_items = int(firstLine[0])
MAX_CAPACITY = int(firstLine[1])

items = []

for i in range(1, total_items+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1]),  int(parts[0])/int(parts[1])))

min_capacity = min([item.weight for item in items])
total_items = len(items)

optimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=True):
    if total_weight-item.weight > 0 :
        optimistic_value += item.value
        total_weight -= item.weight  
    else:
        optimistic_value += total_weight * item.value_per_weight
        break

optimistic_value = int(optimistic_value)

pessimistic_value = 0
total_weight = MAX_CAPACITY
for item in sorted(items, key=lambda x: x.value_per_weight, reverse=False):
    if total_weight-item.weight > 0 :
        pessimistic_value += item.value
        total_weight -= item.weight  
    else:
        pessimistic_value += total_weight * item.value_per_weight
        break
        
pessimistic_value = int(pessimistic_value)     
# min_capacity = MAX_CAPACITY - total_weight + int(total_weight/item.value_per_weight)

total_weight = MAX_CAPACITY
max_items=0
for item in sorted(items, key=lambda x: x.weight, reverse=False):
    if total_weight-item.weight > 0 :
        total_weight -= item.weight  
        max_items+=1
    else:
        break

print('max_capacity: ', MAX_CAPACITY)
print('min_capacity: ', min_capacity)
print('optimistic_value: ', optimistic_value)
print('pessimistic_value: ', pessimistic_value)
print('total_items: ', total_items)
print('max_items: ', max_items)
## 99798
## 100236

max_capacity:  11
min_capacity:  3
optimistic_value:  21
pessimistic_value:  19
total_items:  4
max_items:  2


In [16]:
items = sorted(items, key=lambda x: x.weight, reverse=False)
items

[Item(index=3, value=4, weight=3, value_per_weight=1.3333333333333333),
 Item(index=0, value=8, weight=4, value_per_weight=2.0),
 Item(index=1, value=10, weight=5, value_per_weight=2.0),
 Item(index=2, value=15, weight=8, value_per_weight=1.875)]

In [17]:
max_value = 0
unexplored_nodes = []
optimal_path = []
optimal_node = None
loop_counter = 0

# unexplored_nodes = queue.PriorityQueue()

left_node, right_node = get_children(0, MAX_CAPACITY, optimistic_value, items[0], items, [])

unexplored_nodes.append(right_node)
unexplored_nodes.append(left_node)

# unexplored_nodes.put((-right_node.estimate, right_node))
# unexplored_nodes.put((-left_node.estimate, left_node))

# # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: (x.branch, x.estimate))
unexplored_nodes

[Node(direction='right', index=3, value=0, weight=11, estimate=21, excluded_paths=[3]),
 Node(direction='left', index=3, value=4, weight=8, estimate=21, excluded_paths=[])]

In [20]:
explore_node = unexplored_nodes.pop()
    # _, explore_node = unexplored_nodes.get()

accessible_items = [item for item in items if item.weight <= explore_node.weight and item.index not in explore_node.excluded_paths and item.index != explore_node.index]
## The above statement excludes node for left exploration as well
# accessible_items = []
# for item in items:
#     if item.weight <= explore_node.weight:
#         if explore_node.direction == 'left':
#             accessible_items.append(item)
#         elif item.index not in explore_node.excluded_paths:
#             accessible_items.append(item)

if not accessible_items:
    if explore_node.value > max_value:
        max_value = explore_node.value
        optimal_node = explore_node
    # continue

if accessible_items:
    next_best_item = accessible_items[0]
    
    left_node, right_node = get_children(explore_node.value, explore_node.weight, explore_node.estimate, next_best_item, items, explore_node.excluded_paths)
    
    loop_counter += 1
    
    print('Exploring: ', explore_node)
    print('Max Value: ', max_value)
    print('Left Child: ', left_node)
    print('Right Child: ', right_node)
    
    if right_node.estimate > max_value:
        unexplored_nodes.append(right_node)   
        # unexplored_nodes.put((-right_node.estimate, right_node))
    else:     
        if right_node.value > max_value:
            max_value = right_node.value
            optimal_node = right_node
            
                    
    if left_node.weight >= min_capacity:
        unexplored_nodes.append(left_node)
        # unexplored_nodes.put((-left_node.estimate, left_node))
    else:     
        if left_node.value > max_value:
            max_value = left_node.value
            optimal_node = left_node
    
    print('Max Value: ', max_value)
    print(unexplored_nodes)
    print('---------------------------------------')

accessible_items

Exploring:  Node(direction='left', index=0, value=12, weight=4, estimate=21, excluded_paths=[])
Max Value:  0
Left Child:  Node(direction='left', index=3, value=16, weight=1, estimate=21, excluded_paths=[])
Right Child:  Node(direction='right', index=3, value=12, weight=4, estimate=21, excluded_paths=[3])
Max Value:  16
[Node(direction='right', index=3, value=0, weight=11, estimate=21, excluded_paths=[3]), Node(direction='right', index=0, value=4, weight=8, estimate=21, excluded_paths=[0]), Node(direction='right', index=3, value=12, weight=4, estimate=21, excluded_paths=[3])]
---------------------------------------


[Item(index=3, value=4, weight=3, value_per_weight=1.3333333333333333)]

In [11]:
optimal_node._replace(excluded_paths=[3])

Node(direction='left', index=3, value=19, weight=0, estimate=21, excluded_paths=[3])

In [ ]:
max_value, loop_counter

In [19]:
items


[Item(index=3, value=4, weight=3, value_per_weight=1.3333333333333333),
 Item(index=0, value=8, weight=4, value_per_weight=2.0),
 Item(index=1, value=10, weight=5, value_per_weight=2.0),
 Item(index=2, value=15, weight=8, value_per_weight=1.875)]

In [ ]:
loop_counter = 0

while len(unexplored_nodes) > 0: #not unexplored_nodes.empty(): #len(unexplored_nodes) > 0:
    
    explore_node = unexplored_nodes.pop()
    # _, explore_node = unexplored_nodes.get()
    
    accessible_items = [item for item in items if item.weight <= explore_node.weight and item.index not in explore_node.excluded_paths and item.index != explore_node.index]
    ## The above statement excludes node for left exploration as well
    # accessible_items = []
    # for item in items:
    #     if item.index == explore_node.index:
    #         continue
    #     if explore_node.direction == 'right' and item.index not in explore_node.excluded_paths:
    #         accessible_items.append(item)
    #     elif explore_node.direction == 'left' and item.weight <= explore_node.weight:
    #         accessible_items.append(item)

    if not accessible_items:
        if explore_node.value > max_value:
            max_value = explore_node.value
            optimal_node = explore_node
        continue
    
    next_best_item = accessible_items[0]
    
    left_node, right_node = get_children(explore_node.value, explore_node.weight, explore_node.estimate, next_best_item, items, explore_node.excluded_paths)
    
    loop_counter += 1
    
    # print('Exploring: ', explore_node)
    # print('Max Value: ', max_value)
    # print('Left Child: ', left_node)
    # print('Right Child: ', right_node)
    
    if right_node.estimate > max_value:
        unexplored_nodes.append(right_node)   
        # unexplored_nodes.put((-right_node.estimate, right_node))
    else:     
        if right_node.value > max_value:
            max_value = right_node.value
            optimal_node = right_node
            
                    
    if left_node.weight >= min_capacity:
        unexplored_nodes.append(left_node)
        # unexplored_nodes.put((-left_node.estimate, left_node))
    else:     
        if left_node.value > max_value:
            max_value = left_node.value
            optimal_node = left_node

    # unexplored_nodes = [node for node in unexplored_nodes if node.estimate > max_value]
    # unexplored_nodes = sorted(unexplored_nodes, key=lambda x: x.estimate, reverse=False)

In [ ]:
max_value, loop_counter

In [ ]:
optimal_node

In [ ]:
items

In [ ]:
optimal_path = [1]*len(items)
ignore_item = False
for item in items:
    if item.index in optimal_node.excluded_paths or ignore_item:
       optimal_path[item.index] = 0

    elif item.index == optimal_node.index:
        ignore_item = True

In [ ]:
optimal_path

In [ ]:
items = sorted(items, key=lambda x: x.index, reverse=False)

In [ ]:
verify_value = 0
verify_weight = 0
items = sorted(items, key=lambda x: x.index, reverse=False)

for idx, flag in enumerate(optimal_path):
    if flag:
        print(items[idx])
        verify_value += items[idx].value
        verify_weight += items[idx].weight
        

In [ ]:
verify_value, verify_weight

In [ ]:
accessible_items = []
for item in items:
    if item.weight <= explore_node.weight:
        if explore_node.direction == 'left':
            accessible_items.append(item)
        elif item.index not in explore_node.excluded_paths:
            accessible_items.append(item)